In [ ]:
library(tidyverse)
library(data.table)
library(plotly) # for interactive ploting
library(DT) # for interactive tabulation
library(broom) # for tidy statistical summaries
library(caret) # for regression performance measures
library(psych) # for pairwise comparisons
library(GGally) # for pairwise comparisons
library(magrittr) # for two-way pipes
library(lindia) # for qqplots

In [ ]:
options(repr.matrix.max.rows=20, repr.matrix.max.cols=15) # for limiting the number of top and bottom rows of tables printed 

In [ ]:
datapath <- "~/data_ad454"

# Logistic Regression

We continue with the realty dataset.

Remember that, we calculated the premium_neigh variable which is premium of the unit price of the property over the median unit price of the neighborhood.

Now we will try to classify the properties into premium and discount

Let's first import the realty dataset:

In [ ]:
realty_data3 <- readRDS(sprintf("%s/rds/06_02_realty_data3.rds", datapath))

In [ ]:
realty_data3

Let's add the binary variable premium, which takes 1 when the premium is above 0, and 0 otherwise

In [ ]:
realty_data3[, premium := as.integer(premium_neigh > 0)]

Let's see the structure:

In [ ]:
realty_data3 %>% str

Now, select some of the variables:

In [ ]:
vars <- c("premium", "esyali", "krediye_uygunluk", "bina_yasi", "kat_sayisi", "kat", realty_data3 %>% keep(is.logical) %>% names)
vars

And assign the subset:

In [ ]:
realty_data4 <- realty_data3 %>% select(all_of(vars)) %>% na.omit

Your tasks are to:

- Partition the data set into 70% train and 30% test sets
- Create and run a logistic regression model to explain premium with all other variables **without intercept**. Note that, the median values are taken as basis for premium, so the classes are nearly equal
- Print the summary of the model. Compare and interpret null and residual deviance values and create a table of the coefficients of the variables that are significant at 5% level
- Calculate the fitted positive case ("1") probabilities from the model and also the fitted classes for the train set with a cut value of 0.5
- Create a confusion matrix. You may use the below code template:

```R
table(actual = actual_classes, fitted = fitted_classes) %>% caret::confusionMatrix(positive = "1")
```
- What are the TP, TN, FP, FN counts? Interpret accuracy, sensitivity and specificity.
- Interpret Kappa (what is the level of class agreement)
- Create a ROC curve and calculate AUC. How far is the model better than pure random guessing?
- Calculate the predicted positive case ("1") probabilities from the model and also the predicted classes for the test set with a cut values of 0.5
- Create a confusion matrix for the test set similar to the one above.
- What are the TP, TN, FP, FN counts? Interpret accuracy, sensitivity and specificity.
- Interpret Kappa (what is the level of class agreement)
- Compare the results from the confusion matrices of the train and test sets. 

# Answer

## Partition the dataset

We will take 60% as train and 40% as test set: 

In [ ]:
#partition data
set.seed(2)
train <- realty_data4[,sample(.I, .N * 0.7)]

In [ ]:
realty_train <- realty_data4[train]
realty_test <- realty_data4[-train]

## Build and train a model

Build a logistic regression model: 

In [ ]:
logit_reg <- glm(`premium` ~ . -1, data = realty_train, family = "binomial")

In [ ]:
summary(logit_reg)

In [ ]:
logit_reg %>% broom::tidy() %>% filter(p.value < 0.05)

## Get fitted values and create confusion matrix

In [ ]:
pred_train <- predict(logit_reg, realty_train, type = "response")

In [ ]:
train_class <- ifelse(pred_train > 0.5, 1, 0)

In [ ]:
table(actual = realty_train$premium, fitted = train_class) %>% caret::confusionMatrix(positive = "1")

- TP: 101
- TN: 96
- FP: 46
- FN: 38

- Accuracy: 0.7 of all cases are correctly classified
- Sensitivity: 0.68 of the fitted positive cases are actually positive
- Specificity: 0.71 of the fitted negative cases are actually negative
- Kappa: 0.445, moderate aggreement

## Create ROC and calculate AUC

In [ ]:
p1 <- data.table(D = realty_train[,premium], M = pred_train) %>%
ggplot(aes(m = M, d = D)) +
    plotROC::geom_roc() +
    plotROC::style_roc(theme = theme_grey)

plotROC::export_interactive_roc(p1) %>% IRdisplay::display_html()

In [ ]:
pROC::auc(realty_train[,premium], pred_train)

Pure random guessing would yield 0.5, perfect classification would yield 1. So the model performance is in between the perfect and pure random performances.

## Get predictions on test set and create a confusion matrix

In [ ]:
pred_test <- predict(logit_reg, realty_test, type = "response")

In [ ]:
test_class <- ifelse(pred_test > 0.5, 1, 0)

In [ ]:
table(actual = realty_test$premium, prediction = test_class) %>% caret::confusionMatrix(positive = "1")

- TP: 40
- TN: 36
- FP: 25
- FN: 20

- Accuracy: 0.62 of all cases are correctly classified
- Sensitivity: 0.62 of the fitted positive cases are actually positive
- Specificity: 0.64 of the fitted negative cases are actually negative
- Kappa: 0.25, fair aggreement

Although the classification performance on test set is below that on the train set, we still have a fair prediction performance.